In [1]:
# 正常语句
from langchain_core.prompts import PromptTemplate
example_prompt = PromptTemplate.from_template("Question: {question}\n{answer}")

In [2]:
# 字典格式的例句，所以后续需要转化语句格式的可以通过字典设置
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [45]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [43]:
example_prompt

PromptTemplate(input_variables=['answer', 'question'], template='Question: {question}\n{answer}')

In [6]:
#文本化的few-shot，非message交流式的，不能直接用于大模型的交流
from langchain_core.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(
    few_shot_prompt.invoke({"input": "Who was the father of Mary Ball Washington?"}).to_string()
)

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

In [51]:
messages=few_shot_prompt.format(input="Who was the father of Mary Ball Washington?")
messages

'Question: Who lived longer, Muhammad Ali or Alan Turing?\n\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\n\nQuestion: When was the founder of craigslist born?\n\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\n\nQuestion: Who was the maternal grandfather of George Washington?\n\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was

In [91]:
#生成专门用于交流的few-shot
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())



ValueError: Unexpected message type: humann. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.

In [56]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [64]:
final_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a wondrous wizard of math.')), FewShotChatMessagePromptTemplate(examples=[{'question': 'Who lived longer, Muhammad Ali or Alan Turing?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n'}, {'question': 'When was the founder of craigslist born?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n'}, {'question': 

In [88]:
import os
from langchain_community.chat_models import ChatZhipuAI
zhipuai_chat = ChatZhipuAI(
        temperature=0.5,
        api_key=os.getenv("ZHIPU_API_KEY"),
        model="glm-4-plus",
    )

In [89]:
# 打印结果
prompt_value = final_prompt.format(input='特朗普年龄比拜登大？')

# Assuming 'zhipuai_chat' is your chat model
response = zhipuai_chat.invoke(prompt_value)
print(response.content)

需要后续问题：是。
后续问题：特朗普的出生年份是多少？
中间答案：唐纳德·特朗普出生于1946年。
后续问题：拜登的出生年份是多少？
中间答案：乔·拜登出生于1942年。
所以最终答案是：否，特朗普年龄比拜登小。


In [90]:
chain = final_prompt | zhipuai_chat
chain.invoke({"input": "乔丹比詹姆斯得分更多？"})

AIMessage(content='需要后续问题：是的。\n后续问题：迈克尔·乔丹职业生涯总得分是多少？\n中间答案：迈克尔·乔丹职业生涯总得分是32,292分。\n后续问题：勒布朗·詹姆斯职业生涯总得分是多少？\n中间答案：勒布朗·詹姆斯职业生涯总得分是超过36,000分（具体数字随时间变化，截至2023年已超过这个数字）。\n所以最终答案是：不，勒布朗·詹姆斯比迈克尔·乔丹得分更多。', response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 384, 'total_tokens': 481}, 'model_name': 'glm-4-plus', 'finish_reason': 'stop'}, id='run-13d6b426-3779-484f-abf4-e68dab2a763e-0')